### Louvain Community Detection Algorithm
Não funcionou pois não se aplica a grafos direcionados :(

In [1]:
import networkx as nx
import community as community_louvain

# define the graph
G = nx.read_gml("./rappers_graph.gml")

# retrun partition as a dict
partition = community_louvain.best_partition(G)

# Export the graph to a GML file
nx.write_gml(G, "rappers_graph_louvain.gml")

print("Graph has been exported to 'rappers_graph_louvain.gml'")

TypeError: Bad graph type, use only non directed graph

### Leiden algorithm
Não funcionou muito bem identificando comunidades, alguns nodes ficaram com muitas comunidades enquanto outros sem nenhuma

In [44]:
import networkx as nx
import leidenalg as la
import igraph as ig

# define the graph
G = nx.read_gml("./rappers_graph.gml")

# Transfer node attributes explicitly to the igraph graph
Gig = ig.Graph(directed=nx.is_directed(G))

# Add nodes and edges to the igraph graph
Gig.add_vertices(list(G.nodes()))
Gig.add_edges(list(G.edges()))

# If your NetworkX graph nodes have names or labels, set them as an igraph vertex attribute
Gig.vs["name"] = list(G.nodes())

# Run the Leiden algorithm
partition = la.find_partition(Gig, la.ModularityVertexPartition)

# Create a mapping from igraph indices to NetworkX node names
node_mapping = {v.index: v["name"] for v in Gig.vs}

# Build the partition dictionary using NetworkX node names
partition_dict = {node_mapping[idx]: part for idx, part in enumerate(partition)}

# Add the community attribute to the NetworkX graph
nx.set_node_attributes(G, partition_dict, name="community")

for node in G.nodes():
    if "community" not in G.nodes[node]:
        G.nodes[node]["community"] = -1  # Use -1 for unassigned nodes

# Export the graph with the corrected community information
nx.write_gml(G, "rappers_graph_louvain_fixed.gml")

print("Graph with corrected community information exported to 'rappers_graph_louvain_fixed.gml'")

print(f"Number of nodes in the graph: {len(G.nodes())}")
print(f"Number of nodes in the partition: {len(partition)}")

Graph with corrected community information exported to 'rappers_graph_louvain_fixed.gml'
Number of nodes in the graph: 150
Number of nodes in the partition: 8


### Infomap

In [21]:
from igraph import Graph
import networkx as nx
import igraph as ig

# define the graph
G = nx.read_gml("./rappers_graph.gml")

# Convert the NetworkX graph to an igraph object
Gig = ig.Graph(directed=True)
Gig.add_vertices(list(G.nodes()))
Gig.add_edges(list(G.edges()))

# Run Infomap
partition = Gig.community_infomap()

# Assign communities back to NetworkX graph
partition_dict = {v["name"]: part for v, part in zip(Gig.vs, partition)}
nx.set_node_attributes(G, partition_dict, "community")

# Export the graph with the corrected community information
nx.write_gml(G, "rappers_graph_infomap.gml")

print("Graph with corrected community information exported to 'rappers_graph_infomap.gml'")

Graph with corrected community information exported to 'rappers_graph_infomap.gml'


### Walktrap

In [45]:
import networkx as nx
import leidenalg as la
import igraph as ig

# define the graph
G = nx.read_gml("./rappers_graph.gml")

# Transfer node attributes explicitly to the igraph graph
Gig = ig.Graph(directed=nx.is_directed(G))

# Add nodes and edges to the igraph graph
Gig.add_vertices(list(G.nodes()))
Gig.add_edges(list(G.edges()))

# If your NetworkX graph nodes have names or labels, set them as an igraph vertex attribute
Gig.vs["name"] = list(G.nodes())

# Run the Walktrap algorithm
partition = Gig.community_walktrap().as_clustering()

# Create a mapping from igraph indices to NetworkX node names
node_mapping = {v.index: v["name"] for v in Gig.vs}

# Build the partition dictionary using NetworkX node names
partition_dict = {}
for i, part in enumerate(partition):
    # Skip empty communities (if any)
    if part:
        for node_idx in part:
            node_name = node_mapping[node_idx]
            partition_dict[node_name] = i  # Assign the community index (i) to the node

# Add the community attribute to the NetworkX graph
nx.set_node_attributes(G, partition_dict, name="community")

# Export the graph with the corrected community information
nx.write_gml(G, "rappers_graph_walktrap.gml")

print("Graph with corrected community information exported to 'rappers_graph_walktrap.gml'")

print(f"Number of nodes in the graph: {len(G.nodes())}")
print(f"Number of communities in the partition: {len(partition)}")


Graph with corrected community information exported to 'rappers_graph_walktrap.gml'
Number of nodes in the graph: 150
Number of communities in the partition: 64


### Girvan Newman

In [37]:
import networkx as nx

# define the graph
G = nx.read_gml("./rappers_graph.gml")

# Run Girvan-Newman
communities = list(nx.algorithms.community.girvan_newman(G))

# Get the first level of communities (i.e., after the first edge removal)
first_community_level = communities[0]

# Assign community labels as a node attribute
for i, community in enumerate(first_community_level):
    for node in community:
        G.nodes[node]['community'] = i  # Assign a community index

# Save the graph as a GML file
nx.write_gml(G, "rappers_graph_girvan.gml")


###  Strongly Connected Components (SCC)

In [43]:
import networkx as nx

# define the graph
G = nx.read_gml("./rappers_graph.gml")

# Find strongly connected components (SCC)
sccs = list(nx.strongly_connected_components(G))

# Add community labels (corresponding to each SCC) to nodes
for i, scc in enumerate(sccs):
    for node in scc:
        G.nodes[node]['community'] = i

# Save the graph with SCC-based community information to GML
nx.write_gml(G, "graph_with_scc_communities.gml")
